In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://www.croma.com/searchB?q=led%20tv%3Arelevance&text=led%20tv')

In [4]:
# for each product link
prod_links = driver.find_elements(By.XPATH, "//div[contains(@class, 'plp-prod-title-rating-cont')]/h3[@class='product-title plp-prod-title 999']/a")

for link in prod_links:
    href = link.get_attribute("href")
    print(href)

https://www.croma.com/croma-109-cm-43-inch-4k-ultra-hd-led-google-tv-with-dolby-audio-2024-model-/p/307847
https://www.croma.com/croma-60-cm-24-inch-hd-ready-led-tv-with-a-plus-grade-panel-2024-model-/p/304445
https://www.croma.com/croma-80-cm-32-inch-hd-ready-google-tv-with-dolby-audio-2024-model-/p/307850
https://www.croma.com/croma-102-cm-40-inch-full-hd-led-smart-google-tv-with-dolby-audio-2024-model-/p/307846
https://www.croma.com/croma-109-cm-43-inch-full-hd-led-smart-tv-with-bezel-less-display/p/273733
https://www.croma.com/croma-140-cm-55-inch-4k-ultra-hd-google-tv-with-dolby-audio-2024-model-/p/307848
https://www.croma.com/lg-80cm-32-inch-hd-ready-led-smart-tv-with-dts-virtual-x/p/228393
https://www.croma.com/croma-80-cm-32-inch-hd-ready-led-tv-with-bezel-less-display/p/301734
https://www.croma.com/croma-80-cm-32-inch-hd-ready-led-linux-tv-with-dolby-audio-2024-model-/p/307849
https://www.croma.com/xiaomi-a-series-80-cm-32-inch-hd-ready-led-smart-google-tv-with-voice-assistant

In [5]:
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

In [7]:
# All extracted product links will be stored here

all_product_links = []

def extract_product_links():
    prod_links = driver.find_elements(By.XPATH, "//div[contains(@class, 'plp-prod-title-rating-cont')]/h3[@class='product-title plp-prod-title 999']/a")
    for link in prod_links:
            href = link.get_attribute("href")
            all_product_links.append(href)
            print(href)
    print(len(all_product_links))
            
while True:
    # Extract product links from the current page
    extract_product_links()

    # Identify the "Load More" button or similar element
    try:
        load_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@class='load-more']"))
        )
        load_more_button.click()
        
        # Wait for new products to load
        WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.XPATH, "//ul[@class='product-list']"))
        )
    except TimeoutException:
        # No more products to load, break the loop
        break

https://www.croma.com/croma-109-cm-43-inch-4k-ultra-hd-led-google-tv-with-dolby-audio-2024-model-/p/307847
https://www.croma.com/croma-60-cm-24-inch-hd-ready-led-tv-with-a-plus-grade-panel-2024-model-/p/304445
https://www.croma.com/croma-80-cm-32-inch-hd-ready-google-tv-with-dolby-audio-2024-model-/p/307850
https://www.croma.com/croma-102-cm-40-inch-full-hd-led-smart-google-tv-with-dolby-audio-2024-model-/p/307846
https://www.croma.com/croma-109-cm-43-inch-full-hd-led-smart-tv-with-bezel-less-display/p/273733
https://www.croma.com/croma-140-cm-55-inch-4k-ultra-hd-google-tv-with-dolby-audio-2024-model-/p/307848
https://www.croma.com/lg-80cm-32-inch-hd-ready-led-smart-tv-with-dts-virtual-x/p/228393
https://www.croma.com/croma-80-cm-32-inch-hd-ready-led-tv-with-bezel-less-display/p/301734
https://www.croma.com/croma-80-cm-32-inch-hd-ready-led-linux-tv-with-dolby-audio-2024-model-/p/307849
https://www.croma.com/xiaomi-a-series-80-cm-32-inch-hd-ready-led-smart-google-tv-with-voice-assistant

In [ ]:
# Create a CSV file to store the data
with open('croma_led_tv5.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Product Name', 'Brand', 'Price', 'Ranking', 'Rating', 'Description', 'Image']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()  # Write the header row
    
    rank=0

    for link in all_product_links:
        driver.get(link)
        rank+=1

        # Wait for the product page to load
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//h1[@class='pd-title pd-title-normal']"))
        )

        # Extract product data
        product_title = driver.find_element(By.XPATH, "//h1[@class='pd-title pd-title-normal']").text
        
        # Find the "View More" button
        view_more_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'btn-viewmore-click')]")) 
        )

        # Click the button
        view_more_button.click()

        # Wait for the brand name element to appear
        brand_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[@class='brand-url-pdp spec-brand-url']|//ul[@class='cp-specification-spec-info']/div/li[@class='cp-specification-spec-details']"))
        )

        # Extract the brand name
        brand = brand_element.text
        
        product_price = driver.find_element(By.XPATH, "//span[@class='amount']").text
        price_numeric = product_price.replace("₹", "").replace(",", "")
        price = float(price_numeric)
        ranking = rank
        try:
            rating = driver.find_element(By.XPATH, "//div[contains(@class, 'cp-rating')]/span/span").text
        except NoSuchElementException:
            rating = "No Rating"

        
        # Find the 'ul' element containing the list items
        ul_element = driver.find_element(By.CSS_SELECTOR, ".cp-keyfeature.pd-eligibility-wrap ul")

        # Find all 'li' elements within the 'ul'
        li_elements = ul_element.find_elements(By.TAG_NAME, "li")

        # Extract text from each 'li' and store in a list
        list_items = []
        for li in li_elements:
            list_items.append(li.text)
        
        # Concatenate the list items into a single string
        desc = "\n".join(list_items)
        
        # Find the image element
        image_element = driver.find_element(By.XPATH, "//img[@id='0prod_img']")

        # Extract the 'src' attribute
        img = image_element.get_attribute("src")

        # Write the data to the CSV file
        writer.writerow({'Product Name': product_title, 'Brand': brand,'Price': price, 'Ranking': ranking, 'Rating': rating, 'Description': [desc], 'Image': img})

# Close the browser
driver.quit()
